---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
# !head -5 Employee_Movie_Choices.txt

In [3]:
def answer_one():
            
    return nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')

In [4]:
# plot_graph(answer_one())

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [5]:
def answer_two(): 
    '''
    Reference: https://networkx.github.io/documentation/networkx-1.11/reference/generated/networkx.algorithms.bipartite.projection.weighted_projected_graph.html
    '''
    edgelist = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
    [edgelist.add_node(node, type='employee') if node in employees else edgelist.add_node(node, type='movie') for node in edgelist.nodes()]
    
    return edgelist

answer_two()

In [6]:
# plot_graph(answer_two())

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [7]:
def answer_three():
    '''
    Reference: https://networkx.github.io/documentation/networkx-1.11/reference/generated/networkx.algorithms.bipartite.projection.weighted_projected_graph.html?highlight=weighted_projected_graph#networkx.algorithms.bipartite.projection.weighted_projected_graph
    '''
    edgelist = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
    [edgelist.add_node(node, type='employee') if node in employees else edgelist.add_node(node, type='movie') for node in edgelist.nodes()]
    
    return bipartite.weighted_projected_graph(edgelist, employees)

answer_three()

In [8]:
# plot_graph(answer_three())

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [11]:
def answer_four():
    '''
    Reference: https://stackoverflow.com/questions/50502038/how-can-i-swap-specific-column-values-in-dataframe
    '''
    edgelist1 = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
    [edgelist1.add_node(node, type='employee') if node in employees else edgelist1.add_node(node, type='movie') for node in edgelist1.nodes()]
    weigheted_prj = bipartite.weighted_projected_graph(edgelist1, employees)
    df1 = pd.DataFrame(weigheted_prj.edges(data=True),columns=['L','R','Film Score'])
    df1 = pd.concat([df1, df1]).reset_index().drop(['index'],axis=1)
    df1.loc[11:,['L','R']] = df1.loc[:10,['R','L']].values
  
    edgelist2 = nx.read_edgelist('Employee_Relationships.txt',data=[('Rel Score', int)])
    df2 = pd.DataFrame(edgelist2.edges(data=True), columns=['L','R','Relaltionship Score'])
    
    df1['Film Score'] = df1['Film Score'].map(lambda x: x['weight'] if type(x)==dict else None)
    df2['Relaltionship Score'] = df2['Relaltionship Score'].map(lambda x: x['Rel Score'])
    
    df = pd.merge(df1,df2,on = ['L','R'],how='right')
    df['Film Score'].fillna(value=0,inplace=True)
    
    return df['Film Score'].corr(df['Relaltionship Score'])

answer_four()

          L        R  Film Score  Relaltionship Score
0    Claude  Georgia         3.0                   90
1      Andy    Pablo         1.0                  -10
2      Andy     Joan         1.0                   30
3      Andy  Georgia         1.0                  -10
4      Andy    Frida         1.0                   20
5      Joan      Lee         3.0                   70
6      Andy      Lee         1.0                  -10
7      Andy   Claude         1.0                    0
8     Pablo  Vincent         1.0                  -20
9     Frida  Vincent         2.0                   60
10    Frida    Pablo         2.0                   50
11     Andy  Vincent         0.0                   20
12   Claude    Frida         0.0                    0
13   Claude     Joan         0.0                    0
14   Claude      Lee         0.0                    0
15   Claude    Pablo         0.0                   10
16   Claude  Vincent         0.0                    0
17    Frida  Georgia        

0.7883962221733474